In [3]:
# import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


#### Preprocessing

In [4]:
DataDir = "./leapGestRecog"
ImgSize = (128, 128)
BatchSize = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

train_generator = train_datagen.flow_from_directory(
    DataDir,
    target_size=ImgSize,
    batch_size=BatchSize,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    DataDir,
    target_size=ImgSize,
    batch_size=BatchSize,
    class_mode='categorical',
    subset='validation'
)

Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


#### Model

In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
epochs = 25
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/25
500/500 [==============================] - 385s 767ms/step - loss: 1.9756 - accuracy: 0.2646 - val_loss: 1.8035 - val_accuracy: 0.3528
Epoch 2/25
500/500 [==============================] - 203s 406ms/step - loss: 1.5179 - accuracy: 0.4378 - val_loss: 1.3705 - val_accuracy: 0.5530
Epoch 3/25
500/500 [==============================] - 140s 280ms/step - loss: 1.1542 - accuracy: 0.5727 - val_loss: 1.0663 - val_accuracy: 0.6233
Epoch 4/25
500/500 [==============================] - 131s 261ms/step - loss: 0.9224 - accuracy: 0.6611 - val_loss: 0.9086 - val_accuracy: 0.7060
Epoch 5/25
500/500 [==============================] - 189s 378ms/step - loss: 0.7558 - accuracy: 0.7167 - val_loss: 0.7264 - val_accuracy: 0.7630
Epoch 6/25
500/500 [==============================] - 591s 1s/step - loss: 0.6512 - accuracy: 0.7579 - val_loss: 0.6638 - val_accuracy: 0.7645
Epoch 7/25
500/500 [==============================] - 268s 534ms/step - loss: 0.5759 - accuracy: 0.7864 - val_loss: 0.7723 - va

#### Evaluating the model on validation data

In [7]:
Loss, Accuarcy = model.evaluate(validation_generator)
print(f"Validation Loss: {Loss}")
print(f"Validation Accuracy: {Accuarcy}")

125/125 [==============================] - 15s 123ms/step - loss: 0.3637 - accuracy: 0.8850
Validation Loss: 0.36366379261016846
Validation Accuracy: 0.8849999904632568


#### Predicting 

In [8]:
def Preprocessing(ImgPath):
    Img = tf.keras.preprocessing.image.load_img(ImgPath, target_size=ImgSize)
    ImgArr = tf.keras.preprocessing.image.img_to_array(Img)
    ImgArr = np.expand_dims(ImgArr, axis=0)
    ImgArr /= 255.0
    return ImgArr

TestImgPath = "./Test/03/03_fist/frame_03_03_0005.png"
TestImg = Preprocessing(TestImgPath)
Pred = model.predict(TestImg)
PredClass = np.argmax(Pred, axis=1)
print(f"Predicted Class: {PredClass}")


1/1 [==============================] - 0s 110ms/step
Predicted Class: [3]


#### Saving the model

In [9]:
model.save('hand_gesture_model.h5')

# Load the model
# model = tf.keras.models.load_model('hand_gesture_model.h5')
